In [17]:
! pip install scikit-learn-intelex


In [18]:
from sklearnex import patch_sklearn
patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


PATH="/kaggle/input/webis-clickbait-22/"
sep='[SEP]'

label_encoder = LabelEncoder()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [19]:
model2,encoder,vectorizer = pickle.load(open("/kaggle/input/svm-dataset/model_svm.pkl","rb"))

In [20]:
label_encoder=encoder

In [21]:

def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        i['text']=i['postText']+sep+i['targetParagraphs']+sep+i['targetTitle']+sep+i['targetDescription']+sep+i['targetKeywords']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y


In [22]:

x_train,y_train = get_split_data(PATH+'train.jsonl')
x_test,y_test = get_split_data(PATH+'test.jsonl',False)
x_val,y_val = get_split_data(PATH+'validation.jsonl',False)


len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [23]:
# len(x_train),len(y_train),len(x_test),len(y_test),len(x_val),len(y_val),len(x_train[0]),len(y_train[0])
x_train.shape,y_train.shape,x_test.shape,y_test.shape,x_val.shape,y_val.shape,len(x_train[10])


((3200,), (3200,), (1000,), (1000,), (800,), (800,), 3518)

In [24]:
type(x_train),type(y_train),type(x_test),type(y_test),type(x_val),type(y_val),type(x_train[0]),type(y_train[0])

(numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.str_,
 numpy.int64)

In [25]:

# param_grid={
#     "vectorizer__stop_words":[None,"english"],
#     'vectorizer__ngram_range':[(1,1),(1,2),(2,2)],
#     "model_svm__C":[0.6,0.8,1.0,1.4,1.8,2.2],
#     'model_svm__kernel':['linear','rbf','sigmoid']
# } 

# tf_idf_pipeline = Pipeline([
#     ('vectorizer',TfidfVectorizer()),
#     ('model_svm',SVC())
# ])

# grid_search = GridSearchCV(tf_idf_pipeline, param_grid, cv=10, n_jobs=-1,verbose=4)
# grid_search.fit(x_train, y_train)
# optimalC = grid_search.best_params_["model_svm__C"]
# optimal_kernel = grid_search.best_params_["model_svm__kernel"]
# optimal_ngram = grid_search.best_params_["vectorizer__ngram_range"]
# optimal_stop_words = grid_search.best_params_["vectorizer__stop_words"]
# print("Optimal value of the regularization constant = ",optimalC)
# print("Optimal value of the kernel = ",optimal_kernel)
# print("Optimal value of ngram_range = ",optimal_ngram)
# print("Optimal value of stop_words = ",optimal_stop_words)


In [26]:

vectorizer = TfidfVectorizer(ngram_range=(2,2),stop_words=None)
x_train =  vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)
x_val = vectorizer.transform(x_val)
print(x_train.shape)
print(x_test.shape)



INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


(3200, 706745)
(1000, 706745)


In [27]:

# svm_classifier = SVC(C=optimalC,kernel=optimal_kernel,max_iter=-1)
# svm_classifier.fit(x_train,y_train)

# y_train_pred = svm_classifier.predict(x_train)
# y_test_pred = svm_classifier.predict(x_test)
# y_val_pred = svm_classifier.predict(x_val)
# print("Training data metrics : ")
# print(accuracy_score(y_train_pred,y_train))

# print("Testing data accuracy")
# print(accuracy_score(y_test,y_test_pred))

# print("Validation data accuracy")
# print(accuracy_score(y_val,y_val_pred))




In [28]:
# print("Training data metrics:")
# print(balanced_accuracy_score(y_train_pred, y_train))
# print(f1_score(y_train_pred, y_train, average='weighted'))
# print(classification_report(y_train_pred, y_train))

# print("Testing data metrics:")
# print(balanced_accuracy_score(y_test_pred, y_test))
# print(f1_score(y_test_pred, y_test, average='weighted'))
# print(classification_report(y_test_pred, y_test))

# print("Validation data metrics:")
# print(balanced_accuracy_score(y_val_pred, y_val))
# print(f1_score(y_val_pred, y_val, average='weighted'))
# print(classification_report(y_val_pred, y_val))

In [29]:
# pickle.dump((svm_classifier,label_encoder,vectorizer),open('model_svm.pkl','wb'))

In [30]:

# x_train =  vectorizer.fit_transform(x_train)
# x_test = vectorizer.transform(x_test)
# x_val = vectorizer.transform(x_val)
# print(x_train.shape)
# print(x_test.shape)



In [31]:
y_train_pred = model2.predict(x_train)
y_test_pred = model2.predict(x_test)
y_val_pred = model2.predict(x_val)
print("Training data metrics : ")
print(accuracy_score(y_train,y_train_pred))

print("Testing data accuracy")
print(accuracy_score(y_test,y_test_pred))

print("Testing data accuracy")
print(accuracy_score(y_val,y_val_pred))

INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Training data metrics : 
0.99875
Testing data accuracy
0.576
Testing data accuracy
0.55875


In [32]:
print("Training data metrics:")
print(balanced_accuracy_score(y_train,y_train_pred))
print(f1_score(y_train,y_train_pred, average='macro'))
print(classification_report(y_train,y_train_pred))

print("Testing data metrics:")
print(balanced_accuracy_score(y_test,y_test_pred))
print(f1_score(y_test, y_test_pred, average='macro'))
print(classification_report(y_test,y_test_pred))

print("Validation data metrics:")
print(balanced_accuracy_score(y_val, y_val_pred))
print(f1_score(y_val,y_val_pred, average='macro'))
print(classification_report(y_val,y_val_pred))

Training data metrics:
0.9983019080723149
0.9986446870118243
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       559
           1       1.00      1.00      1.00      1274
           2       1.00      1.00      1.00      1367

    accuracy                           1.00      3200
   macro avg       1.00      1.00      1.00      3200
weighted avg       1.00      1.00      1.00      3200

Testing data metrics:
0.5234841633322215
0.5374121146015354
              precision    recall  f1-score   support

           0       0.64      0.30      0.41       174
           1       0.56      0.62      0.59       403
           2       0.58      0.64      0.61       423

    accuracy                           0.58      1000
   macro avg       0.59      0.52      0.54      1000
weighted avg       0.58      0.58      0.57      1000

Validation data metrics:
0.5093634248079414
0.5217385295743505
              precision    recall  f1-score   support
